In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "data/Motor___MDS-UPDRS/"
PATH_DAT = "./"

In [4]:
import pydicom
import glob
import os
import numpy

In [5]:
up11 = pd.read_csv(f'{PATH}MDS_UPDRS_Part_I__Patient_Questionnaire.csv')
up12 = pd.read_csv(f'{PATH}MDS_UPDRS_Part_I.csv')

In [6]:
up2 = pd.read_csv(f'{PATH}MDS_UPDRS_Part_II__Patient_Questionnaire.csv')

In [7]:
up3 = pd.read_csv(f'{PATH}MDS_UPDRS_Part_III__Post_Dose_.csv')

In [8]:
up4 = pd.read_csv(f'{PATH}MDS_UPDRS_Part_IV.csv')

In [9]:
up11['TOTAL11'] = (up11.iloc[:,7:14]).sum(axis=1)
up12['TOTAL12'] = (up12.iloc[:,7:14]).sum(axis=1)
up2['TOTAL2'] = (up12.iloc[:,7:20]).sum(axis=1)
up3['TOTAL3'] = (up3.iloc[:,7:41]).sum(axis=1)
up4['TOTAL4'] = (up4.iloc[:,6:12]).sum(axis=1)

In [10]:
#up3.loc[up3['EVENT_ID']=='BL',"EVENT_ID"] = 'SC'

In [11]:
up3 = up3[~up3["PAG_NAME"].str.contains('A')]

In [12]:
up11['index'] = up11["PATNO"].map(str)+up11["EVENT_ID"].map(str)
#up3.set_index('index',inplace=True)
up11 = up11[['PATNO','index','TOTAL11']]

In [13]:
up12['index'] = up12["PATNO"].map(str)+up12["EVENT_ID"].map(str)
#up3.set_index('index',inplace=True)
up12 = up12[['TOTAL12','index']]

In [14]:
up2['index'] = up2["PATNO"].map(str)+up2["EVENT_ID"].map(str)
#up3.set_index('index',inplace=True)
up2 = up2[['TOTAL2','index']]

In [15]:
up3['index'] = up3["PATNO"].map(str)+up3["EVENT_ID"].map(str)
#up3.set_index('index',inplace=True)
up3 = up3[['TOTAL3','index']]

In [16]:
up4['index'] = up4["PATNO"].map(str)+up4["EVENT_ID"].map(str)
#up3.set_index('index',inplace=True)
up4 = up4[['TOTAL4','index']]

In [115]:
up2.loc[up2['PATNO']=='3762']

KeyError: 'PATNO'

In [116]:
up3.loc[up3['PATNO']=='3762']

KeyError: 'PATNO'

In [17]:
combine1to4 = pd.merge(up11,up12, how='inner', on='index')

In [18]:
combine1to4 = pd.merge(combine1to4,up2, how='inner', on='index')

In [19]:
combine1to4 = pd.merge(combine1to4,up4, how='inner', on='index')

In [20]:
combine1to4 = pd.merge(combine1to4,up3, how='inner', on='index')

In [21]:
combine1to4['TOTAL'] = (combine1to4.iloc[:,2:7]).sum(axis=1)

In [22]:
combine1to4

,PATNO,index,TOTAL11,TOTAL12,TOTAL2,TOTAL4,TOTAL3,TOTAL
0,3403,3403ST,4.0,2.0,8.0,0.0,42.0,56.0
1,3251,3251U01,13.0,4.0,2.0,0.0,27.0,46.0
2,3400,3400V04,14.0,7.0,0.0,0.0,44.0,65.0
3,3800,3800ST,3.0,0.0,0.0,0.0,10.0,13.0
4,3403,3403V04,9.0,1.0,2.0,0.0,45.0,57.0
5,3406,3406V04,7.0,2.0,4.0,0.0,29.0,42.0
6,3006,3006V01,0.0,2.0,0.0,0.0,30.0,32.0
7,3502,3502V03,8.0,0.0,2.0,0.0,41.0,51.0
8,3054,3054V03,5.0,0.0,6.0,1.0,15.0,27.0
9,4006,4006PW,2.0,0.0,0.0,0.0,39.0,41.0


In [129]:
combine1to4.to_csv('Total_score_UPDRS.csv')

In [23]:
dat = pd.read_csv(f'{PATH_DAT}PPMI.csv')

In [24]:
#Change Visit code to standard code
dat.loc[dat['Visit']==0,"Visit"] = 'SC'
dat.loc[dat['Visit']==5,"Visit"] = 'V04'
dat.loc[dat['Visit']==7,"Visit"] = 'V06'
dat.loc[dat['Visit']==9,"Visit"] = 'V09'
dat.loc[dat['Visit']==11,"Visit"] = 'V10'
dat.loc[dat['Visit']==13,"Visit"] = 'V13'
dat.loc[dat['Visit']==21,"Visit"] = 'V21'
dat.loc[dat['Visit']==90,"Visit"] = 'U01'
dat.loc[dat['Visit']==91,"Visit"] = 'U02'
dat.loc[dat['Visit']==92,"Visit"] = 'U03'

In [25]:
dat["Visit"].unique()

array(['SC', 'V04', 'V06', 'V10', 'V21', 'V09', 'U01', 'U03', 'V13', 'U02'], dtype=object)

In [26]:
dat['index'] = dat["Subject"].map(str)+dat["Visit"].map(str)
dat = dat[['Group','index','Image Data ID']]
#dat.set_index('index',inplace=True)

In [30]:
dat_analysis = pd.read_csv('data/DaTSCAN/DATScan_Analysis.csv')

In [31]:
dat_analysis['EVENT_ID'].unique()

array(['SC', 'V10', 'V06', 'V04', 'ST', 'U01', 'U02'], dtype=object)

In [27]:
combine1to4 = pd.merge(combine1to4, dat, how='inner', on='index')

In [28]:
combine1to4.head()

,PATNO,index,TOTAL11,TOTAL12,TOTAL2,TOTAL4,TOTAL3,TOTAL,Group,Image Data ID
0,3400,3400V04,14.0,7.0,0.0,0.0,44.0,65.0,PD,355944
1,3403,3403V04,9.0,1.0,2.0,0.0,45.0,57.0,PD,419562
2,3406,3406V04,7.0,2.0,4.0,0.0,29.0,42.0,PD,355945
3,3051,3051V04,3.0,1.0,2.0,1.0,16.0,23.0,PD,355935
4,3500,3500V04,9.0,2.0,0.0,0.0,29.0,40.0,PD,355950


In [31]:
lstFilesDCM =[]
for file in glob.glob(f'{PATH_DAT}PPMI*/**/Reconstructed_DaTSCAN/**/**/*.dcm'):
    #file = file.split("_")[13].strip('I').strip('.dcm')
    lstFilesDCM.append(file)

In [32]:
len(lstFilesDCM)

1569

In [33]:
stripLst = []
for x in lstFilesDCM:
    x = x.split("_")[13].strip('I').strip('.dcm')
    stripLst.append(x)

In [34]:
combine1to4 = combine1to4.set_index('Image Data ID')
for index, rows in combine1to4.iterrows():
    for i in lstFilesDCM:
        j = i.split("_")[13].strip()
        j = int(j[1:7])
        if index == j:
            combine1to4.loc[combine1to4.index == j, "Filenames"] = i
combine1to4

,PATNO,index,TOTAL11,TOTAL12,TOTAL2,TOTAL4,TOTAL3,TOTAL,Group,Filenames
Image Data ID,,,,,,,,,,
355944,3400,3400V04,14.0,7.0,0.0,0.0,44.0,65.0,PD,./PPMI 4/3400/Reconstructed_DaTSCAN/2011-07-05...
419562,3403,3403V04,9.0,1.0,2.0,0.0,45.0,57.0,PD,./PPMI 8/3403/Reconstructed_DaTSCAN/2011-07-13...
355945,3406,3406V04,7.0,2.0,4.0,0.0,29.0,42.0,PD,./PPMI 5/3406/Reconstructed_DaTSCAN/2011-07-28...
355935,3051,3051V04,3.0,1.0,2.0,1.0,16.0,23.0,PD,./PPMI 5/3051/Reconstructed_DaTSCAN/2011-11-08...
355950,3500,3500V04,9.0,2.0,0.0,0.0,29.0,40.0,PD,./PPMI 2/3500/Reconstructed_DaTSCAN/2011-12-08...
355948,3451,3451V04,3.0,0.0,0.0,0.0,25.0,28.0,PD,./PPMI/3451/Reconstructed_DaTSCAN/2011-12-21_1...
355937,3054,3054V04,5.0,1.0,2.0,0.0,27.0,35.0,PD,./PPMI 6/3054/Reconstructed_DaTSCAN/2011-12-13...
358124,3415,3415V04,6.0,0.0,2.0,0.0,36.0,44.0,PD,./PPMI 2/3415/Reconstructed_DaTSCAN/2012-04-13...
355941,3252,3252V04,2.0,0.0,6.0,1.0,24.0,33.0,PD,./PPMI 9/3252/Reconstructed_DaTSCAN/2011-12-21...


In [35]:
combine1to4 = combine1to4[combine1to4.index.isin(stripLst)]

In [36]:
combine1to4.to_csv('combine1to4.csv')